# Projeto 1 - Ciência dos Dados

Nome: Rafael Roismann

Nome: Rafael Curie Raiz

Atenção: Serão permitidos grupos de três pessoas, mas com uma rubrica mais exigente. Grupos deste tamanho precisarão fazer um questionário de avaliação de trabalho em equipe

___
Carregando algumas bibliotecas:

In [159]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [160]:
print('Esperamos trabalhar no diretório')
print(os.getcwd())

Esperamos trabalhar no diretório
/Users/Rafa/Insper/2 Semestre/CDados/Projeto-01


Carregando a base de dados com os tweets classificados manualmente:

In [161]:
train = pd.read_excel('dados_treino.xlsx')
train.head(5)

#train.tail(5)

,Mensagem,Classificacao
0,Um livro para ler e jogar fora e sem comentar ...,1
1,"Varios personagens sem desfecho, final sem pé ...",1
2,Realmente. Deixou muito a desejar. Esperei um ...,1
3,"amei o livro inteiro, mas confesso que o final...",1
4,A tradução é péssima e diferente do original. ...,0


In [162]:
test = pd.read_excel('dados_teste.xlsx')
test.head(5)

,Mensagem,Classificacao
0,"O que foi isto que recebi aqui em casa, mas di...",0
1,O livro é uma mera propagando de leitura dinâm...,1
2,"A história pode ser boa, a leitura é muito can...",1
3,Achei que o livro mostra apenas oque o senso c...,1
4,"Não gostei muito do livro, ele é todo escrito ...",1


___
## Classificador automático


Abaixo foi criado um classificador segundo a teoria de probabilidade de *Naive Bayes*, nessa classificação foram utilizados os dois critérios abaixo para a classificação das frases:

* **Critica sobre conteúdo do livro:** Dentro do escopo de nossa análise, todos os comentários que mencionam o autor da obra ou oferecem alguma forma de avaliação em relação ao seu conteúdo no livro foram cuidadosamente categorizados sob a classificação '1'. Essa classificação abrange uma ampla gama de comentários que exploram diversos aspectos da obra, desde a qualidade da narrativa até as críticas construtivas e as opiniões pessoais dos leitores. Ao examinarmos os comentários, consideramos não apenas as observações objetivas relacionadas ao livro, como sua trama, personagens e estilo de escrita, mas também as percepções subjetivas dos leitores em relação às expectativas que tinham antes de ler o livro e as eventuais frustrações que possam ter surgido durante a leitura.

* **Criticas Sobre serviço da amazom e relacionados:** Esse item refere-se às frases previamente identificadas como comentários relativos ao serviço, ou a qualquer aspecto relacionado, como a entrega do produto ou a qualidade da tradução do livro. Essas frases abrangem as opiniões, observações e feedback dos clientes em relação à experiência que tiveram com o serviço em questão, bem como quaisquer serviços complementares, como a entrega ou a tradução do livro.

___
### Montando um Classificador Naive-Bayes

Considerando apenas as mensagens da planilha Treinamento, ensine  seu classificador.

#### 1- Limpando o DataFrame

In [163]:
def cleanup(text):

    letras = 'abcdefghijklmnopqrstuvwxyz úáãâõôóàçé'+'abcdefghijklmnopqrstuvwxyz úáãâõôóàçé'.upper()
    texto = []
    for i in list(text):
        if i in list(letras):
            texto.append(i)
    return ''.join(texto).lower()
print(cleanup('oi! tudo bem?'))


oi tudo bem


#### 2 - Fitrando as frases

Na seção abaixo, usaremos a função acima para limpar o Dataframe e filtramos cada frase de acordo com sua classificação em relação a classificação pré definida anterioremnte, o Target do projeto.

In [164]:
# Criando as listas que armazenarão as palavras das frases nos respectivos grupos

# Grupo 0 - Críticas sobre conteúdo do livro
list_conteudo = []
# Grupo 1 - Criticas Sobre serviço da amazom e relacionados
list_servico = []

for i in range(0,300):

    # Pegar cada frase dentro do DataFrame e fazer a limpeza e transformar cada palavra em um valor de uma lista 
    list_palavra = cleanup(train.Mensagem[i]).split()
    if train.Classificacao[i] == 0:    # Caso não seja classificado como críticas sobre o conteúdo do livro
        for j in list_palavra:            
            list_servico.append(j)

    if train.Classificacao[i] == 1:    # Caso não seja classificado como críticas sobre o conteúdo do livro
        for j in list_palavra:
            list_conteudo.append(j)
            
# list_conteudo = [i for i in list_conteudo if i not in ['do', 'e', 'a', 'o', 'não', 'um', 'que']]
# list_servico = [i for i in list_conteudo if i not in ['do', 'e', 'a', 'o', 'não', 'um', 'que']]
print(list_conteudo[:10])


['um', 'livro', 'para', 'ler', 'e', 'jogar', 'fora', 'e', 'sem', 'comentar']


### 3 - Criando a segunda limpagem dos dados

### 4 - Criando um pd.Series e criando a tabela de frequência absoluta

Criando um pd.Series com as palavras classificadas como ***críticas ao livro*** e criando a sua respectiva tabela de frequência absoluta

In [165]:
# Guardando as palavras em um pd.Series
serie_conteudo = pd.Series(list_conteudo)
# Tabela de frequência absoluta
tabela_conteudo = serie_conteudo.value_counts()
tabela_conteudo.head(10)


de       370
o        349
e        346
que      343
a        307
não      251
livro    221
é        169
um       165
do       142
dtype: int64

Criando um pd.Series com as palavras classificadas como ***críticas sobre o serviço da amazom e relacionados*** e criando a sua respectiva tabela de frequência absoluta

In [166]:
# Guardando as palavras em um pd.Series
serie_servico = pd.Series(list_servico)

# Tabela de frequência absoluta
tabela_servico = serie_servico.value_counts()
tabela_servico.head(10)

o        174
a        171
de       159
que      146
e        144
não       86
livro     68
com       64
em        61
do        53
dtype: int64

Criando um pd.series com todas as palavras e criando a sua tabela de frequência absoluta

In [167]:
# Todas as palavras dos comentários
total_palavras = list_conteudo + list_servico
palavras_unicas = len(set(total_palavras))

# Guardando as palavras em um pd.Series
serie_total = pd.Series(total_palavras)
print(len(serie_conteudo)/len(serie_total), len(serie_servico)/len(serie_total) )
# Tabela de frequência absoluta
tabela_total = serie_total.value_counts(True)
tabela_total.head(10)


0.6863264020163831 0.3136735979836169


de       0.033333
o        0.032955
e        0.030876
que      0.030813
a        0.030120
não      0.021235
livro    0.018210
é        0.013926
um       0.013674
do       0.012287
dtype: float64

In [168]:
print([1,2,3,4]+[3,4,5,6])

[1, 2, 3, 4, 3, 4, 5, 6]


___
### Verificando a performance do Classificador

Agora você deve testar o seu classificador com a base de Testes.

In [169]:
def testar():
    acerto_Cont = 0
    erros_Cont = 0
    acerto_Serv = 0
    erros_Serv = 0
    lenlist_servico = len(list_servico)
    lenlist_conteudo = len(list_conteudo)
    for i in range(200):
        frase = cleanup(test.Mensagem[i])
        vdd = test.Classificacao[i] # verdadeira classificação
        probServ = len(list_servico)/len(total_palavras) # probabilidade inicial de ser serviço
        probCont = len(list_conteudo)/len(total_palavras) # probabilidade inicial de ser conteudo
        lista_frase = frase.split()
        for palavra in lista_frase:
            try:
                contagemServ_palavra = tabela_servico[palavra] 
            except:
                contagemServ_palavra = 0 # caso a palavra não esteja na tabela
            try:
                contagemCont_palavra = tabela_conteudo[palavra]
            except:
                contagemCont_palavra = 0 # caso a palavra não esteja na tabela

            prob_servico = (contagemServ_palavra+1)/(lenlist_servico+palavras_unicas) # Suavizador Laplace
            prob_conteudo = (contagemCont_palavra+1)/(lenlist_conteudo+palavras_unicas )

            probServ *= prob_servico # Multiplicando as probabilidades
            probCont *= prob_conteudo
            if probServ == 0 or probCont == 0: # Caso a multiplicacao seja tão pequena que o computador não consiga calcular, ele para de calcular e faz o chute
                chute = not probServ > probCont 
                break
        probCont *=1.074
        chute = not probServ > probCont # Verificando qual probabilidade é maior
        if chute == vdd:
            if vdd:  
                acerto_Cont += 1
            else:
                acerto_Serv += 1
        else:
            if vdd:
                erros_Serv += 1
            else:
                erros_Cont += 1
    precisao_conteudo = acerto_Cont/(acerto_Cont+erros_Cont)
    precisao_servico = acerto_Serv/(acerto_Serv+erros_Serv)
    RealCont = acerto_Cont / (acerto_Cont + erros_Serv) 
    ReaclServ = acerto_Serv / (acerto_Serv+erros_Cont)
    accuracy = (acerto_Cont + acerto_Serv) / 200
    print(f"quanto % dos conteudos acertou {round(precisao_conteudo, 3)*100}%")
    print(f"quanto % dos servicos acertou {round(precisao_servico, 3)*100}%")
    print(f"quanto % dos conteudos eram realmete conteudos {round(RealCont, 4)*100}%")
    print(f"quanto % dos servicos eram realmete servicos {round(ReaclServ, 4)*100}%")
    print(f"quanto % dos chutes estavam certos {round(accuracy, 3)*100}%")
"""
melhor = 0
melhor_alpha = 0
for i in range(900, 1200):
    alpha = i/1000
    accuracy = testar(alpha)
    if accuracy > melhor:          # encontrando o melhor alpha. melhor encontrado = 1.074
        melhor = accuracy        
        melhor_alpha = alpha
print(melhor, melhor_alpha)        """
testar()

quanto % dos conteudos acertou 93.2%
quanto % dos servicos acertou 88.9%
quanto % dos conteudos eram realmete conteudos 95.77%
quanto % dos servicos eram realmete servicos 82.76%
quanto % dos chutes estavam certos 92.0%


#### 2 - Coluna 'Type'

Criando a coluna ***Type*** responsável por guardar as classificações

In [170]:
def Classifica(frase):
    palavras_unicas = len(set(total_palavras))
    probServ = 1
    probCont = 1
    lista_frase = frase.split()
    lenlist_servico = len(list_servico)
    lenlist_conteudo = len(list_conteudo)
    for palavra in lista_frase:
        contagemServ_palavra = tabela_servico[palavra] if palavra in list_servico else 0
        contagemCont_palavra = tabela_conteudo[palavra] if palavra in list_conteudo else 0
        prob_servico = (contagemServ_palavra+1)/(lenlist_servico+palavras_unicas) # Suavizador Laplace
        prob_conteudo = (contagemCont_palavra+1)/(lenlist_conteudo+palavras_unicas )
        probServ *= prob_servico
        probCont *= prob_conteudo
        if probServ == 0 or probCont == 0:
            chute = not probServ > probCont
            return chute
            
    return not probServ > probCont

In [171]:

test['Type'] = [Classifica(test['Mensagem'][i]) for i in range(200)]

___
### Concluindo

In [172]:
def ClassificaFrase(frase):
    palavras_unicas = len(set(total_palavras))
    probServ = 1
    probCont = 1
    lista_frase = frase.split()
    lenlist_servico = len(list_servico)
    lenlist_conteudo = len(list_conteudo)
    for palavra in lista_frase:
        contagemServ_palavra = tabela_servico[palavra] if palavra in list_servico else 0
        contagemCont_palavra = tabela_conteudo[palavra] if palavra in list_conteudo else 0
        prob_servico = (contagemServ_palavra+1)/(lenlist_servico+palavras_unicas) # Suavizador Laplace
        prob_conteudo = (contagemCont_palavra+1)/(lenlist_conteudo+palavras_unicas )
        probServ *= prob_servico
        probCont *= prob_conteudo
        if probServ == 0 or probCont == 0:
            chute = not probServ > probCont
            return chute, 2
    chute = not probServ > probCont
    if chute:
        confianca = probServ/probCont
        if confianca < 0.2: confianca = 'alta'
        elif confianca < 0.8: confianca = 'media'
        else: confianca = 'baixa'
        return 'conteudo', confianca
    else:
        confianca = probCont/probServ
        if confianca < 0.2: confianca = 'alta'
        elif confianca < 0.8: confianca = 'media'
        else: confianca = 'baixa'
        return 'servico', confianca
print(ClassificaFrase('a capa do livro veio amassada'))

('servico', 'alta')


___
### Qualidade do Classificador a partir de novas separações dos tweets entre Treinamento e Teste

Caso for fazer esse item do Projeto

NameError: name 'lis' is not defined

___
## Aperfeiçoamento:

Trabalhos que conseguirem pelo menos conceito B vão evoluir em conceito dependendo da quantidade de itens avançados:

* IMPLEMENTOU outras limpezas e transformações que não afetem a qualidade da informação contida nos tweets. Ex: stemming, lemmatization, stopwords
* CONSIDEROU mais de duas categorias na variável Target e INCREMENTOU a quantidade de notícias, mantendo pelo menos 250 notícias por categoria (OBRIGATÓRIO PARA TRIOS, sem contar como item avançado)
* Para Target com duas categorias: CRIOU pelo menos quatro categorias intermediárias de relevância baseadas na probabilidade: ex.: muito relevante, relevante, neutro, irrelevante, muito irrelevante
* EXPLICOU porquê não pode usar o próprio classificador para gerar mais amostras de treinamento
* PROPÔS diferentes cenários para Naïve Bayes fora do contexto do projeto (pelo menos dois cenários, exceto aqueles já apresentados em sala pelos professores: por exemplo, filtro de spam)
* SUGERIU e EXPLICOU melhorias reais com indicações concretas de como implementar (indicar como fazer e indicar material de pesquisa)
* FEZ o item Qualidade do Classificador a partir de novas separações das Notícias entre Treinamento e Teste descrito no enunciado do projeto (OBRIGATÓRIO para conceitos A ou A+)

___
## Referências

[Naive Bayes and Text Classification](https://arxiv.org/pdf/1410.5329.pdf)  **Mais completo**

[A practical explanation of a Naive Bayes Classifier](https://monkeylearn.com/blog/practical-explanation-naive-bayes-classifier/) **Mais simples**